In [1]:
%pylab qt
%load_ext autoreload
%autoreload 2

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
# Loop through the ggems mhd files and save them to an array
import glob
import fastcat as fc
import numpy as np
import matplotlib.pyplot as plt
import mpl_interactions as pli
from mpl_interactions import ipyplot as iplt
# Zoom the ggems projections
from scipy.ndimage import zoom

ggems_scatter_files = glob.glob(
    '/home/jericho/1-Workspace/opengate_ggems_comparison/2-fastcat_scatter_XCAT/test/out/ggems_3e09_121kVp_*scatter.mhd'
)
ggems_primary_files = glob.glob(
    '/home/jericho/1-Workspace/opengate_ggems_comparison/2-fastcat_scatter_XCAT/test/out/ggems_3e09_121kVp_*[!scatter].mhd'
)

ggems_scatter_files.sort()
ggems_primary_files.sort()

ggems_primary_projections = []
ggems_scatter_projections = []

for ggems_scatter_file, ggems_primary_file in zip(ggems_scatter_files, ggems_primary_files):
    ggems_prmiary, b, c = fc.utils.read_mhd(ggems_primary_file)
    ggems_scatter, b, c = fc.utils.read_mhd(ggems_scatter_file)

    ggems_primary_projections.append(ggems_prmiary.squeeze())
    ggems_scatter_projections.append(ggems_scatter.squeeze())

# Load the pickled phantom
pkl_file = '/home/jericho/1-Workspace/opengate_ggems_comparison/2-fastcat_scatter_XCAT/test/ggems_1e03_121kVp.pkl'
phantom = np.load(pkl_file, allow_pickle=True)

phantom.simulation_parameters['angles'] = phantom.angles 
# Convert the phantom angles to degrees
phantom_angles = phantom.angles * 180 / np.pi

# Define the denoising functions
def downsize_block(image, block_size):
    image_size = image.shape[0]
    block_size = int(block_size)
    downsize = image_size//block_size
    image = image[:downsize*block_size, :downsize*block_size]
    image = image.reshape(downsize, block_size, downsize,
                          block_size).mean(axis=(1, 3))
    return image


def denoise_projections(projections):
    projections_zoomed = []

    for projection in projections:
        projection_zoomed_downsized = downsize_block(projection, 64)
        projection_zoomed = zoom(
            projection_zoomed_downsized, 64, order=4, mode='nearest')
        # Adjust the mean of the zoomed image to match the original
        projection_zoomed = projection_zoomed * \
            np.mean(projection) / np.mean(projection_zoomed)
        projections_zoomed.append(projection_zoomed)

    return projections_zoomed


# zoom the scatter projections using the zoom fromfunction
ggems_scatter_denoised = denoise_projections(ggems_scatter_projections)



In [3]:
angles = phantom.angles * 180 / np.pi

# Estimate the scatter at the angle opposite the detector as the fliplr of the scatter at a given angle
ggems_scatter_denoised_flipped = [
    np.fliplr(projection) for projection in ggems_scatter_denoised]
ggems_scatter_opposite = [ggems_scatter_denoised_flipped[round(
    abs(180-angle)/(360/len(angles)))] for angle in angles]

In [7]:
# append the first half of the array fliped left to right and concatenate the two arrays
ggems_scatter_denoised_opposite = []
for ii, ggems_scatter in enumerate(ggems_scatter_denoised):
    if ii < 45:
        ggems_scatter_denoised_opposite.append(ggems_scatter_denoised[ii])
    else:
        ggems_scatter_denoised_opposite.append(
            np.fliplr(ggems_scatter_denoised[ii % 45]))

# append the first half of the array fliped left to right and concatenate the two arrays
ggems_projection_denoised_opposite = []
for ii, ggems_scatter in enumerate(ggems_primary_projections):
    if ii < 45:
        ggems_projection_denoised_opposite.append(
            ggems_primary_projections[ii])
    else:
        ggems_projection_denoised_opposite.append(
            np.fliplr(ggems_primary_projections[ii % 45]))

In [34]:
fig = plt.figure()
plt.subplot(321)
controls = pli.hyperslicer(ggems_scatter_denoised,
                           autoscale_cmap=False, cmap='turbo')
plt.title('Scatter [Counts]')
plt.subplot(323)
new = pli.hyperslicer(ggems_scatter_denoised_opposite,
                      controls=controls, autoscale_cmap=False, cmap='turbo')
plt.title('Scatter opposite [Counts]')
plt.subplot(322)
new = pli.hyperslicer(ggems_projection_denoised_opposite,
                      controls=controls, autoscale_cmap=False, cmap='turbo')
plt.title('Projection opposite [Counts]')
plt.subplot(324)
new = pli.hyperslicer(ggems_primary_projections,
                      controls=controls, autoscale_cmap=False, cmap='turbo')
plt.title('Projection [Counts]')

# Turn off all the subplot axes
for ax in plt.gcf().axes:
    ax.axis('off')
# plot the difference between the scatter and the opposite scatter
ggems_scatter_denoised_opposite = np.array(ggems_scatter_denoised_opposite)
ggems_scatter_denoised = np.array(ggems_scatter_denoised)
# Calculate the difference between the scatter and the opposite scatter in percentage
ggems_scatter_difference = (
    ggems_scatter_denoised - ggems_scatter_denoised_opposite)/np.max(ggems_scatter_denoised)*100

plt.subplot(325)
new = pli.hyperslicer(ggems_scatter_difference, autoscale_cmap=False,
                      cmap='bwr', controls=controls, vmin=-10, vmax=10)
plt.axis('off')
plt.title('Scatter difference [%]')
plt.colorbar()

# plot the difference between the projection and the opposite projection
ggems_projection_denoised_opposite = np.array(
    ggems_projection_denoised_opposite)
ggems_primary_projections = np.array(ggems_primary_projections)
# Calculate the difference between the scatter and the opposite scatter in percentage
ggems_projection_difference = (
    ggems_primary_projections - ggems_projection_denoised_opposite)/np.max(ggems_primary_projections)*100

plt.subplot(326)
new = pli.hyperslicer(ggems_projection_difference, autoscale_cmap=False,
                      cmap='bwr', controls=controls, vmin=-10, vmax=10)
plt.axis('off')
plt.title('Projection difference [%]')
plt.colorbar()
plt.tight_layout()

[2023-12-12 15:32:31,574] {colorbar.py:859} DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x7f7005bb82d0>
[2023-12-12 15:32:31,694] {colorbar.py:859} DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x7f7005c40ed0>


In [35]:
anim = controls.save_animation(
    "phantoms_ggems_analytical+denoised_opposite.mp4", fig, "axis0", interval=120)

[2023-12-12 15:32:46,728] {animation.py:1060} INFO - Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
[2023-12-12 15:32:46,729] {animation.py:318} INFO - figure size in inches has been adjusted from 8.12 x 9.64 to 8.1 x 9.64
[2023-12-12 15:32:46,729] {animation.py:322} DEBUG - frame size in pixels is 810 x 964
[2023-12-12 15:32:46,730] {animation.py:338} INFO - MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 810x964 -pix_fmt rgba -framerate 8.333333333333334 -i pipe: -vcodec h264 -pix_fmt yuv420p -y phantoms_ggems_analytical+denoised_opposite.mp4
[2023-12-12 15:32:46,734] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2023-12-12 15:32:46,850] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2023-12-12 15:32:46,956] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2023-12-12 15:32:47,063] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2023-12-12 15:32:47,167] {animat

In [98]:
from scipy import ndimage


def interpolate_images(images, new_depth):
    # Calculate the zoom factor
    zoom_factor = new_depth / images.shape[2]

    # Use scipy.ndimage.zoom to interpolate the images
    interpolated_images = ndimage.zoom(images, (4, 1, 1), order=1)

    return interpolated_images


def interpolate_each_pixel(image, z0, z1):
    # loop over the pixels and interpolate the images
    image_large = np.zeros((len(z1), image.shape[1], image.shape[2]))
    for ii in range(image.shape[1]):
        for jj in range(image.shape[2]):
            image_large[:, ii, jj] = np.interp(z1, z0, image[:, ii, jj])

    return image_large


def normalize_mean_and_std(array1,array2):
    array1 = array1 - np.mean(array1)
    # array1 = array1 * np.std(array2) / np.std(array1)
    array1 = array1 + np.mean(array2)
    return array1

ggems_scatter_norm = normalize_mean_and_std(ggems_scatter_denoised, ggems_scatter_projections)

# Test the function
images = np.array(ggems_scatter_norm[::4])
z0 = np.arange(0, 90, 4)
z1 = np.arange(0, 90, 1)

interpolated_images = interpolate_each_pixel(images, z0, z1)
# interpolated_images = interpolate_images(images, 2)

In [107]:
# Calculate the error between the interpolated images and the original images
error = np.abs(interpolated_images - np.array(ggems_scatter_norm)
               )/np.max(np.array(ggems_scatter_norm))*100

plt.figure()
plt.plot(np.mean(error, axis=(1, 2)),'b*')
plt.title('Mean error [%]')
plt.xlabel('Angle')
plt.ylabel('RMSE [%]')
# add a red line for the mean of the points that are not zero
error[error == 0] = np.nan
plt.axhline(y=np.nanmean(np.nanmean(error, axis=(1, 2))), color='r', linestyle='-')
# label the line with it's value
plt.text(0, np.nanmean(np.nanmean(error, axis=(1, 2))), 'mean = {:.2f}'.format(np.nanmean(np.nanmean(error, axis=(1, 2)))))

plt.savefig('ggems_scatter_interpolation_error.png', dpi=300)

/tmp/ipykernel_6741/732142513.py:12: RuntimeWarning: Mean of empty slice
  plt.axhline(y=np.nanmean(np.nanmean(error, axis=(1, 2))), color='r', linestyle='-')
/tmp/ipykernel_6741/732142513.py:14: RuntimeWarning: Mean of empty slice
  plt.text(0, np.nanmean(np.nanmean(error, axis=(1, 2))), 'mean = {:.2f}'.format(np.nanmean(np.nanmean(error, axis=(1, 2)))))


In [100]:
fig = plt.figure(figsize=(10, 10))
plt.subplot(221)
controls = pli.hyperslicer(ggems_scatter_projections,
                           autoscale_cmap=False, cmap='turbo', play_buttons=True,
                           vmin=np.percentile(interpolated_images, 1), vmax=np.percentile(interpolated_images, 99))
plt.title('Scatter [Counts]')
plt.subplot(223)
new = pli.hyperslicer(interpolated_images, controls=controls, autoscale_cmap=False, cmap='turbo',
                      vmin=np.percentile(interpolated_images, 1), vmax=np.percentile(interpolated_images, 99))
plt.title('Scatter interpolated [Counts]')
plt.subplot(222)
new = pli.hyperslicer(ggems_primary_projections,
                      controls=controls, autoscale_cmap=False, cmap='turbo')
plt.title('Projection [Counts]')

# Turn off all the subplot axes
for ax in plt.gcf().axes:
    ax.axis('off')

# Calculate the difference between the scatter and the interpolated scatter in percentage
ggems_scatter_difference = (
    ggems_scatter_projections - interpolated_images)/(ggems_scatter_norm)*100

ggems_scatter_diff_small = []
for ii in range(len(ggems_scatter_difference)):
    ggems_scatter_diff_small.append(
        downsize_block(ggems_scatter_difference[ii], 32))

plt.subplot(224)
new = pli.hyperslicer(ggems_scatter_diff_small, autoscale_cmap=False,
                      cmap='bwr', controls=controls, vmin=-5, vmax=5)
plt.axis('off')
plt.title('Scatter difference [%]')
plt.colorbar()
plt.tight_layout()

[2023-12-12 16:55:48,886] {colorbar.py:859} DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x7f7000536d50>


In [101]:
anim = controls.save_animation(
    "phantoms_ggems_analytical+denoised_interpolated_15 projections.mp4", fig, "axis0", interval=120)

[2023-12-12 16:58:25,277] {animation.py:1060} INFO - Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
[2023-12-12 16:58:25,277] {animation.py:322} DEBUG - frame size in pixels is 1000 x 964
[2023-12-12 16:58:25,278] {animation.py:338} INFO - MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 1000x964 -pix_fmt rgba -framerate 8.333333333333334 -i pipe: -vcodec h264 -pix_fmt yuv420p -y 'phantoms_ggems_analytical+denoised_interpolated_15 projections.mp4'
[2023-12-12 16:58:25,280] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2023-12-12 16:58:25,366] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2023-12-12 16:58:25,448] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2023-12-12 16:58:25,531] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2023-12-12 16:58:25,613] {animation.py:366} DEBUG - MovieWriter.grab_frame: Grabbing frame.
[2023-12-12 16:58:25,696] {animation.py: